![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/07.1.Pretrained_NER_Profiling_Pipelines.ipynb)

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/5.0.0.spark_nlp_for_healthcare.json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.0.0, 💊Spark-Healthcare==5.0.0, running on ⚡ PySpark==3.1.2


# 🔎 Pipelines

<div align="center">

| **Index** | **NER Profiling Pipelines**        |
|---------------|----------------------|
| 1        | [ner_profiling_clinical](https://nlp.johnsnowlabs.com/2022/08/30/ner_profiling_clinical_en.html)     |
| 2          | [ner_profiling_biobert](https://nlp.johnsnowlabs.com/2023/06/17/ner_profiling_biobert_en.html)       |
| 3      | [ner_profiling_oncology](https://nlp.johnsnowlabs.com/2023/07/03/ner_profiling_oncology_en.html)    |
| 4      | [ner_profiling_sdoh](https://nlp.johnsnowlabs.com/2023/07/11/ner_profiling_sdoh_en.html)    |
| 5      | [ner_profiling_vop](https://nlp.johnsnowlabs.com/2023/07/03/ner_profiling_vop_en.html)    |
</div>

# Clinical NER Model Profiling Pretrained Pipeline

This pipeline can be used to explore all the available pretrained NER models at once. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with `embeddings_clinical`.

<center><b>NER Profiling Clinical Model List</b></center>


|| | | |
|--------------|-----------------|-----------------|-----------------|
| jsl_ner_wip_clinical | jsl_ner_wip_greedy_clinical | jsl_ner_wip_modifier_clinical | jsl_rd_ner_wip_greedy_clinical |
| ner_abbreviation_clinical | ner_ade_binary | ner_ade_clinical | ner_anatomy |
| ner_anatomy_coarse | ner_bacterial_species | ner_biomarker | ner_biomedical_bc2gm |
| ner_bionlp | ner_cancer_genetics | ner_cellular | ner_chemd_clinical |
| ner_chemicals | ner_chemprot_clinical | ner_chexpert | ner_clinical |
| ner_clinical_large | ner_clinical_trials_abstracts | ner_covid_trials | ner_deid_augmented |
| ner_deid_enriched | ner_deid_generic_augmented | ner_deid_large | ner_deid_sd |
| ner_deid_sd_large | ner_deid_subentity_augmented | ner_deid_subentity_augmented_i2b2 | ner_deid_synthetic |
| ner_deidentify_dl | ner_diseases | ner_diseases_large | ner_drugprot_clinical |
| ner_drugs | ner_drugs_greedy | ner_drugs_large | ner_events_admission_clinical |
| ner_events_clinical | ner_genetic_variants | ner_human_phenotype_gene_clinical | ner_human_phenotype_go_clinical |
| ner_jsl | ner_jsl_enriched | ner_jsl_greedy | ner_jsl_slim |
| ner_living_species | ner_measurements_clinical | ner_medmentions_coarse | ner_nature_nero_clinical |
| ner_nihss | ner_pathogen | ner_posology | ner_posology_experimental |
| ner_posology_greedy | ner_posology_large | ner_posology_small | ner_radiology |
| ner_radiology_wip_clinical | ner_risk_factors | ner_supplement_clinical | nerdl_tumour_demo |

You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [ ]:
document_assembler = nlp.DocumentAssembler()\
      .setInputCol('text')\
      .setOutputCol('document')

sentence_detector = nlp.SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
      .setInputCols("sentence")\
      .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

#NER model to detect drug in the text
ner = medical.NerModel.pretrained('ner_posology_greedy', 'en', 'clinical/models') \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_chunk = medical.NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

chunkMapper = medical.ChunkMapperModel.pretrained("drug_ade_mapper", "en", "clinical/models")\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("mappings")\
      .setRels(["ADE"])

pipeline = nlp.Pipeline().setStages([document_assembler,
                                 sentence_detector,
                                 tokenizer,
                                 word_embeddings,
                                 ner,
                                 ner_chunk,
                                 chunkMapper])

text = ["""The patient was prescribed 1000 mg fish oil and multivitamins.
            She was discharged on zopiclone and ambrisentan"""]

test_data = spark.createDataFrame([text]).toDF("text")
model = pipeline.fit(test_data)
res= model.transform(test_data)
res.show()

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_posology_greedy download started this may take some time.
[OK!]
drug_ade_mapper download started this may take some time.
[OK!]
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|            mappings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|The patient was p...|[{document, 0, 12...|[{document, 0, 61...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 27, 42, ...|[{labeled_depende...|
+--------------------+--------------------+--------------------+--------------------

In [ ]:
clinical_profiling_pipeline = nlp.PretrainedPipeline("ner_profiling_clinical", "en", "clinical/models")

ner_profiling_clinical download started this may take some time.
Approx size to download 2.9 GB
[OK!]


In [ ]:
text = '''A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting .'''

In [ ]:
clinical_result = clinical_profiling_pipeline.annotate(text)
clinical_result.keys()

dict_keys(['ner_vop_problem_wip_chunks', 'ner_sdoh_demographics_wip_chunks', 'ner_ade_clinical_chunks', 'ner_deid_augmented', 'ner_deid_subentity_augmented_i2b2', 'ner_posology_greedy_chunks', 'ner_vop_demographic_wip_chunks', 'ner_sdoh_community_condition_wip_chunks', 'ner_radiology_wip_clinical', 'ner_vop_slim_wip', 'ner_oncology_diagnosis_chunks', 'ner_vop_test_wip_chunks', 'ner_jsl_slim', 'ner_vop_wip', 'ner_risk_factors_chunks', 'jsl_ner_wip_clinical_chunks', 'ner_oncology_unspecific_posology_chunks', 'ner_vop_problem_reduced_wip', 'ner_vop_anatomy_wip', 'ner_vop_problem_reduced_wip_chunks', 'ner_eu_clinical_case_chunks', 'ner_oncology_test_chunks', 'ner_deid_synthetic', 'ner_oncology_posology_chunks', 'ner_oncology_tnm_chunks', 'ner_oncology_anatomy_general_chunks', 'ner_drugs_greedy', 'ner_abbreviation_clinical_chunks', 'ner_covid_trials_chunks', 'ner_human_phenotype_gene_clinical_chunks', 'ner_events_admission_clinical', 'jsl_ner_wip_greedy_clinical_chunks', 'ner_posology_greed

In [ ]:
print("Clinical Text: \n", text, "\n \nResults:\n")

for i in clinical_result.keys():
  print("{} : ".format(i), clinical_result[i])

Clinical Text: 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
 
Results:

ner_vop_problem_wip_chunks :  ['gestational diabetes mellitus', 'diabetes mellitus', 'T2DM', 'HTG-induced pancreatitis', 'acute', 'hepatitis', 'obesity', 'polyuria', 'polydipsia', 'poor appetite', 'vomiting']
ner_sdoh_demographics_wip_chunks :  ['28-year-old', 'female']
ner_ade_clinical_chunks :  ['polydipsia', 'poor appetite', 'vomiting']
ner_deid_augmented :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

**Lets show chunk results of NER models in a pandas dataframe.**

In [ ]:
import pandas as pd


def get_chunk_results(light_result):

  for i in light_result.keys():

    model_name = "_".join(i.split("_")[:-1])

    if (i == 'sentence') | (i == 'token'):

      continue

    if ("_chunks" not in i):

      continue

    elif len(light_result[i]) == 0:

      print("\n", "*"*20, model_name, "Model Results", "*"*20)
      print("No Result For This Model")


    else:

      sentences = []
      begins = []
      ends = []
      chunks = []
      entities = []
      confidences = []

      for n in (light_result[i]):

        sentences.append(n.metadata['sentence'])
        begins.append(n.begin)
        ends.append(n.end)
        chunks.append(n.result)
        entities.append(n.metadata['entity'])
        confidences.append(n.metadata['confidence'])


      df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'chunks': chunks, 'entity': entities, 'confidence': confidences})

      print("\n \n", "*"*20, model_name, "Model Results", "*"*20)
      display(df)

In [ ]:
text2 = """
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma.
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy.
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%.
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007
"""

In [ ]:
clinical_full_result = clinical_profiling_pipeline.fullAnnotate(text2)[0]

In [ ]:
clinical_full_result['ner_jsl_chunks']

[Annotation(chunk, 15, 34, chest tube placement, {'entity': 'Procedure', 'sentence': '0', 'chunk': '0', 'confidence': '0.5311666'}, []),
 Annotation(chunk, 75, 86, thoracoscopy, {'entity': 'Procedure', 'sentence': '0', 'chunk': '1', 'confidence': '0.9944'}, []),
 Annotation(chunk, 93, 106, fluid biopsies, {'entity': 'Procedure', 'sentence': '0', 'chunk': '2', 'confidence': '0.7353'}, []),
 Annotation(chunk, 146, 179, epithelioid malignant mesothelioma, {'entity': 'Oncological', 'sentence': '0', 'chunk': '3', 'confidence': '0.87376666'}, []),
 Annotation(chunk, 219, 224, PET CT, {'entity': 'Test', 'sentence': '1', 'chunk': '4', 'confidence': '0.71175003'}, []),
 Annotation(chunk, 240, 248, extensive, {'entity': 'Modifier', 'sentence': '1', 'chunk': '5', 'confidence': '0.6863'}, []),
 Annotation(chunk, 250, 268, uptake in the chest, {'entity': 'Symptom', 'sentence': '1', 'chunk': '6', 'confidence': '0.526375'}, []),
 Annotation(chunk, 271, 279, bilateral, {'entity': 'Direction', 'sentenc

In [ ]:
print("Clinical Text:\n", text2)

Clinical Text:
 
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007



In [ ]:
get_chunk_results(clinical_full_result)


 
 ******************** ner_vop_problem_wip Model Results ********************


sentence  begin  end                              chunks    entity  \
0        0    146  179  epithelioid malignant mesothelioma   Disease   
1        1    301  309                           effusions   Disease   
2        1    316  330                     lymphadenopathy   Disease   
3        2    347  352                              acidic  Modifier   
4        2    354  358                               fluid   Symptom   

   confidence  
0  0.83923334  
1      0.4851  
2      0.9492  
3      0.6155  
4       0.793


 
 ******************** ner_sdoh_demographics_wip Model Results ********************


sentence  begin  end chunks  entity confidence
0        2    334  336    She  Gender        1.0
1        4    475  477    Her  Gender     0.9999
2        5    545  547    She  Gender        1.0


 
 ******************** ner_ade_clinical Model Results ********************


...

sentence  begin  end            chunks        entity confidence
0        5    568  583  Oncology service  ClinicalDept    0.85605


 
 ******************** ner_ade_binary Model Results ********************


sentence  begin  end                                   chunks entity  \
0        0    146  179       epithelioid malignant mesothelioma    ADE   
1        1    250  268                      uptake in the chest    ADE   
2        1    271  309  bilateral pleural pericardial effusions    ADE   
3        1    316  330                          lymphadenopathy    ADE   
4        2    347  358                             acidic fluid    ADE   
5        2    361  397    pectoral and intramammary lymph nodes    ADE   
6        2    403  423                    uptake in L4 with SUV    ADE   
7        3    462  472                              III disease    ADE   
8        4    511  527                        ejection fraction    ADE   

  confidence  
0     0.6943  
1     0.5401  
2   0.610125  
3     0.9985  
4     0.6749  
5    0.72288  
6    0.76716  
7    0.54185  
8    0.72225


 
 ******************** ner_events_admission_clinical Model Results ********************


sentence  begin  end                                             chunks  \
0         0     15   34                               chest tube placement   
1         0     40   60                              drainage of the fluid   
2         0     75   86                                       thoracoscopy   
3         0     93  106                                     fluid biopsies   
4         0    137  144                                           revealed   
5         0    146  179                 epithelioid malignant mesothelioma   
6         1    204  210                                            stained   
7         1    217  224                                           a PET CT   
8         1    233  238                                             showed   
9         1    240  268                      extensive uptake in the chest   
10        1    271  309            bilateral pleural pericardial effusions   
11        1    316  330                                    lymphadenopathy   
12        2    347  397  acidic fluid, pectoral and intramammary lymph ...   
13        2    403  414                                       uptake in L4   
14        2    421  423                                                SUV   
15        3    456  472                                  stage III disease   
16        4    475  499                          Her repeat echocardiogram   
17        4    501  506                                             showed   
18        4    508  527                               an ejection fraction   
19        5    553  563                                        transferred   
20        5    568  583                                   Oncology service   
21        5    600  611                                       chemotherapy   
22        5    616  632                                  September 1, 2007   
23        5    639  647                                          cisplatin   
24        5    703  714                                       over 2 hours   
25        5    719  735                                  September 1, 2007   

           entity  confidence  
0       TREATMENT   0.8781667  
1       TREATMENT      0.4451  
2            TEST      0.9948  
3            TEST  0.84274995  
4      EVIDENTIAL      0.5703  
5         PROBLEM  0.87450004  
6         PROBLEM      0.3036  
7            TEST  0.74953336  
8      EVIDENTIAL      0.9502  
9         PROBLEM     0.73456  
10        PROBLEM     0.84565  
11        PROBLEM      0.9673  
12        PROBLEM  0.60776246  
13        PROBLEM   0.5861667  
14           TEST      0.9866  
15        PROBLEM      0.8269  
16           TEST   0.6921666  
17     EVIDENTIAL      0.9871  
18     OCCURRENCE  0.40440002  
19     OCCURRENCE      0.9965  
20  CLINICAL_DEPT      0.7772  
21      TREATMENT       0.994  
22           DATE    0.925325  
23      TREATMENT      0.9982  
24      FREQUENCY  0.72436666  
25           DATE     0.74775

**Now we will show all NER labels of tokens in the same dataframe.**

In [ ]:
def get_token_results(light_result):

  tokens = [j.result for j in light_result["token"]]
  sentences = [j.metadata["sentence"] for j in light_result["token"]]
  begins = [j.begin for j in light_result["token"]]
  ends = [j.end for j in light_result["token"]]
  model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

  df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

  for model_name in model_list:

    temp_df = pd.DataFrame(light_result[model_name])
    temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
    temp_df = temp_df[["jsl_label"]]

    # temp_df = get_ner_result(model_name)
    temp_df.columns = [model_name]
    df = pd.concat([df, temp_df], axis=1)

  return df

In [ ]:
get_token_results(clinical_full_result)

sentence  begin  end      token ner_deid_augmented  \
0          0      1    2         At                  O   
1          0      4    7       this                  O   
2          0      9   12       time                  O   
3          0     13   13          ,                  O   
4          0     15   19      chest                  O   
..       ...    ...  ...        ...                ...   
123        5    716  717         on                  O   
124        5    719  727  September             B-DATE   
125        5    729  729          1             I-DATE   
126        5    730  730          ,             I-DATE   
127        5    732  735       2007             I-DATE   

    ner_deid_subentity_augmented_i2b2 ner_radiology_wip_clinical  \
0                                   O                          O   
1                                   O                          O   
2                                   O                          O   
3                                   O                          O   
4                                   O                B-Procedure   
..                                ...                        ...   
123                                 O                          O   
124                            B-DATE                          O   
125                            I-DATE                          O   
126                            I-DATE                          O   
127                            I-DATE                          O   

    ner_vop_slim_wip ner_jsl_slim  ner_vop_wip  ... ner_events_clinical  \
0                  O            O            O  ...                   O   
1                  O            O            O  ...              B-DATE   
2                  O            O            O  ...              I-DATE   
3                  O            O            O  ...                   O   
4        B-Procedure  B-Procedure  B-Procedure  ...         B-TREATMENT   
..               ...          ...          ...  ...                 ...   
123                O            O            O  ...                   O   
124       B-DateTime  B-Date_Time   B-DateTime  ...              B-DATE   
125       I-DateTime  I-Date_Time   I-DateTime  ...              I-DATE   
126       I-DateTime  I-Date_Time   I-DateTime  ...              I-DATE   
127       I-DateTime  I-Date_Time   I-DateTime  ...              I-DATE   

    ner_supplement_clinical ner_oncology_unspecific_posology  \
0                         O                                O   
1                         O                                O   
2                         O                                O   
3                         O                                O   
4                         O                                O   
..                      ...                              ...   
123                       O                                O   
124                       O                                O   
125                       O                                O   
126                       O                                O   
127                       O                                O   

    ner_genetic_variants ner_sdoh_wip ner_radiology ner_eu_clinical_case  \
0                      O            O             O                    O   
1                      O            O             O                    O   
2                      O            O             O     B-clinical_event   
3                      O            O             O                    O   
4                      O            O   B-Procedure           B-bodypart   
..                   ...          ...           ...                  ...   
123                    O            O             O          I-date_time   
124                    O            O             O          I-date_time   
125                    O            O             O          I-date_time   
126                    O            O             O         

# BioBert NER Model Profiling Pretrained Pipeline

This pipeline can be used to explore all the available pretrained NER models at once. When you run this pipeline over your text, you will end up with the predictions coming out of each pretrained clinical NER model trained with `biobert_pubmed_base_cased`.

<center><b>NER Profiling BioBert Model List</b>


| | |
|-|-|
| ner_cellular_biobert           | ner_clinical_biobert             |
| ner_diseases_biobert           | ner_anatomy_coarse_biobert       |
| ner_events_biobert             | ner_human_phenotype_gene_biobert |
| ner_bionlp_biobert             | ner_posology_large_biobert       |
| ner_jsl_greedy_biobert         | jsl_rd_ner_wip_greedy_biobert    |
| ner_jsl_biobert                | ner_posology_biobert             |
| ner_anatomy_biobert            | jsl_ner_wip_greedy_biobert       |
| ner_jsl_enriched_biobert       | ner_chemprot_biobert             |
| ner_human_phenotype_go_biobert | ner_ade_biobert                  |
| ner_deid_biobert               | ner_risk_factors_biobert         |
| ner_deid_enriched_biobert      | ner_living_species_biobert                                |



</center>

You can check [Models Hub](https://nlp.johnsnowlabs.com/models) page for more information about all these models and more.

In [ ]:
biobert_profiling_pipeline = nlp.PretrainedPipeline("ner_profiling_biobert", "en", "clinical/models")

ner_profiling_biobert download started this may take some time.
Approx size to download 731.1 MB
[OK!]


In [ ]:
bio_result = biobert_profiling_pipeline.annotate(text)
bio_result.keys()

dict_keys(['ner_ade_biobert', 'ner_cellular_biobert_chunks', 'ner_human_phenotype_go_biobert', 'ner_events_biobert', 'ner_diseases_biobert_chunks', 'ner_events_biobert_chunks', 'jsl_ner_wip_greedy_biobert', 'ner_bionlp_biobert_chunks', 'ner_bionlp_biobert', 'ner_jsl_greedy_biobert_chunks', 'ner_jsl_biobert_chunks', 'ner_jsl_biobert', 'ner_anatomy_biobert_chunks', 'ner_posology_large_biobert', 'ner_jsl_enriched_biobert_chunks', 'jsl_rd_ner_wip_greedy_biobert', 'ner_human_phenotype_go_biobert_chunks', 'ner_posology_biobert', 'ner_deid_biobert_chunks', 'ner_jsl_greedy_biobert', 'ner_chemprot_biobert', 'ner_deid_enriched_biobert_chunks', 'ner_jsl_enriched_biobert', 'token', 'ner_deid_biobert', 'ner_clinical_biobert_chunks', 'ner_human_phenotype_gene_biobert', 'ner_anatomy_coarse_biobert_chunks', 'ner_human_phenotype_gene_biobert_chunks', 'ner_posology_large_biobert_chunks', 'jsl_rd_ner_wip_greedy_biobert_chunks', 'ner_posology_biobert_chunks', 'ner_clinical_biobert', 'ner_diseases_biobert'

In [ ]:
print("Clinical Text: \n", text, "\n \nResults:\n")

for i in bio_result.keys():
  print("{} : ".format(i), bio_result[i])

Clinical Text: 
 A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . 
 
Results:

ner_ade_biobert :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE', 'I-ADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE', 'O', 'B-ADE', 'O', 'B-ADE', 'I-ADE', 'O', 'O', 'B-ADE', 'O']
ner_cellular_biobert_chunks :  []
ner_human_phenotype_go_biobert :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [ ]:
bio_full_result = biobert_profiling_pipeline.fullAnnotate(text2)[0]

In [ ]:
bio_full_result['ner_jsl_biobert_chunks']

[Annotation(chunk, 15, 34, chest tube placement, {'entity': 'Procedure', 'sentence': '0', 'chunk': '0', 'confidence': '0.5898667'}, []),
 Annotation(chunk, 40, 60, drainage of the fluid, {'entity': 'Symptom', 'sentence': '0', 'chunk': '1', 'confidence': '0.28855002'}, []),
 Annotation(chunk, 75, 86, thoracoscopy, {'entity': 'Procedure', 'sentence': '0', 'chunk': '2', 'confidence': '0.9924'}, []),
 Annotation(chunk, 93, 106, fluid biopsies, {'entity': 'Procedure', 'sentence': '0', 'chunk': '3', 'confidence': '0.76275'}, []),
 Annotation(chunk, 146, 179, epithelioid malignant mesothelioma, {'entity': 'Oncological', 'sentence': '0', 'chunk': '4', 'confidence': '0.53603333'}, []),
 Annotation(chunk, 219, 224, PET CT, {'entity': 'Test', 'sentence': '1', 'chunk': '5', 'confidence': '0.71395004'}, []),
 Annotation(chunk, 240, 255, extensive uptake, {'entity': 'ImagingFindings', 'sentence': '1', 'chunk': '6', 'confidence': '0.5604'}, []),
 Annotation(chunk, 264, 268, chest, {'entity': 'Externa

In [ ]:
print("Clinical Text:\n", text2)

Clinical Text:
 
At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, which were performed, which revealed epithelioid malignant mesothelioma. 
The patient was then stained with a PET CT, which showed extensive uptake in the chest, bilateral pleural pericardial effusions, and lymphadenopathy. 
She also had acidic fluid, pectoral and intramammary lymph nodes and uptake in L4 with SUV of 4. This was consistent with stage III disease. Her repeat echocardiogram showed an ejection fraction of 45% to 49%. 
She was transferred to Oncology service and started on chemotherapy on September 1, 2007 with cisplatin 75 mg/centimeter squared equaling 109 mg IV piggyback over 2 hours on September 1, 2007



**Show NER chunks in a pandas dataframe.**

In [ ]:
get_chunk_results(bio_full_result)


 ******************** ner_cellular_biobert Model Results ********************
No Result For This Model

 
 ******************** ner_diseases_biobert Model Results ********************


sentence  begin  end                              chunks   entity  \
0        0    146  179  epithelioid malignant mesothelioma  Disease   
1        1    281  309       pleural pericardial effusions  Disease   
2        1    316  330                     lymphadenopathy  Disease   
3        3    456  472                   stage III disease  Disease   

   confidence  
0  0.95236665  
1   0.8773667  
2      0.9807  
3   0.8513667


 
 ******************** ner_events_biobert Model Results ********************


sentence  begin  end                                             chunks  \
0         0     15   34                               chest tube placement   
1         0     40   47                                           drainage   
2         0     52   60                                          the fluid   
3         0     75   86                                       thoracoscopy   
4         0     93  106                                     fluid biopsies   
5         0    137  144                                           revealed   
6         0    146  179                 epithelioid malignant mesothelioma   
7         1    217  224                                           a PET CT   
8         1    233  238                                             showed   
9         1    240  268                      extensive uptake in the chest   
10        1    271  309            bilateral pleural pericardial effusions   
11        1    316  330                                    lymphadenopathy   
12        2    347  397  acidic fluid, pectoral and intramammary lymph ...   
13        2    403  414                                       uptake in L4   
14        2    421  423                                                SUV   
15        3    456  472                                  stage III disease   
16        4    475  499                          Her repeat echocardiogram   
17        4    501  506                                             showed   
18        4    508  527                               an ejection fraction   
19        5    553  563                                        transferred   
20        5    568  583                                   Oncology service   
21        5    600  611                                       chemotherapy   
22        5    616  632                                  September 1, 2007   
23        5    639  647                                          cisplatin   
24        5    708  714                                            2 hours   
25        5    719  735                                  September 1, 2007   

           entity  confidence  
0       TREATMENT  0.82250005  
1       TREATMENT      0.9375  
2         PROBLEM     0.92555  
3            TEST      0.6154  
4            TEST     0.88645  
5      EVIDENTIAL      0.9861  
6         PROBLEM  0.93763334  
7            TEST  0.94936675  
8      EVIDENTIAL      0.9169  
9         PROBLEM     0.82536  
10        PROBLEM    0.876475  
11        PROBLEM      0.8454  
12        PROBLEM  0.73123753  
13        PROBLEM      0.7737  
14           TEST      0.9839  
15        PROBLEM  0.76570004  
16           TEST  0.92633337  
17     EVIDENTIAL      0.9836  
18           TEST  0.63166666  
19     OCCURRENCE      0.9887  
20  CLINICAL_DEPT     0.88925  
21      TREATMENT      0.9914  
22           DATE    0.814475  
23      TREATMENT      0.9957  
24       DURATION     0.63945  
25           DATE  0.67277503


 
 ******************** ner_bionlp_biobert Model Results ********************


...

sentence  begin  end                                   chunks entity  \
0        0     15   34                     chest tube placement   DRUG   
1        0     40   60                    drainage of the fluid   DRUG   
2        0     75   86                             thoracoscopy   DRUG   
3        1    240  268            extensive uptake in the chest    ADE   
4        1    271  309  bilateral pleural pericardial effusions    ADE   
5        2    347  358                             acidic fluid    ADE   
6        3    456  460                                    stage   DRUG   
7        5    600  611                             chemotherapy   DRUG   
8        5    639  647                                cisplatin   DRUG   
9        5    693  701                                piggyback   DRUG   

   confidence  
0   0.9891667  
1  0.77637494  
2      0.9375  
3     0.90334  
4     0.73015  
5      0.6747  
6      0.9395  
7      0.9912  
8       0.992  
9      0.8437


 
 ******************** ner_risk_factors_biobert Model Results ********************


sentence  begin  end     chunks entity confidence
0        5    616  624  September    PHI     0.9707
1        5    719  727  September    PHI     0.9858
2        5    732  735       2007    PHI     0.9828


 
 ******************** ner_living_species_biobert Model Results ********************


sentence  begin  end   chunks entity confidence
0        1    187  193  patient  HUMAN     0.9994

**Show NER label results in a pandas dataframe.**

In [ ]:
get_token_results(bio_full_result)

sentence  begin  end      token ner_ade_biobert  \
0          0      1    2         At               O   
1          0      4    7       this               O   
2          0      9   12       time               O   
3          0     13   13          ,               O   
4          0     15   19      chest          B-DRUG   
..       ...    ...  ...        ...             ...   
123        5    716  717         on               O   
124        5    719  727  September               O   
125        5    729  729          1               O   
126        5    730  730          ,               O   
127        5    732  735       2007               O   

    ner_human_phenotype_go_biobert ner_events_biobert  \
0                                O                  O   
1                                O                  O   
2                                O                  O   
3                                O                  O   
4                                O        B-TREATMENT   
..                             ...                ...   
123                              O                  O   
124                              O             B-DATE   
125                              O             I-DATE   
126                              O             I-DATE   
127                              O             I-DATE   

    jsl_ner_wip_greedy_biobert        ner_bionlp_biobert ner_jsl_biobert  ...  \
0                            O                         O               O  ...   
1                            O                         O               O  ...   
2                            O                         O               O  ...   
3                            O                         O               O  ...   
4                  B-Procedure  B-Multi-tissue_structure     B-Procedure  ...   
..                         ...                       ...             ...  ...   
123                          O                         O               O  ...   
124                     B-Date                         O          B-Date  ...   
125                     I-Date                         O          I-Date  ...   
126                     I-Date                         O          I-Date  ...   
127                     I-Date                         O          I-Date  ...   

    ner_deid_biobert ner_human_phenotype_gene_biobert ner_clinical_biobert  \
0                  O                                O                    O   
1                  O                                O                    O   
2                  O                                O                    O   
3                  O                                O                    O   
4                  O                             B-HP          B-TREATMENT   
..               ...                              ...                  ...   
123                O                                O                    O   
124           B-DATE                                O                    O   
125           I-DATE                                O                    O   
126           I-DATE                                O                    O   
127           I-DATE                                O                    O   

    ner_diseases_biobert ner_living_species_biobert  \
0                      O                          O   
1                      O                          O   
2                      O                          O   
3                      O                          O   
4                      O                          O   
..                   ...                        ...   
123                    O                          O   
124                    O                          O   
125                    O                          O   
126                    O                          O   
127                    O                          O   

    ner_anatomy_coarse_biobert ner_cellular_biobert ner_anatomy_biobert  \
0           

# Oncology Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once.

- `ner_profiling_oncology` : Returns results for oncology NER models.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).





<center><b>NER Profiling Oncology Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| ner_oncology_unspecific_posology | ner_oncology_tnm | ner_oncology_therapy | ner_oncology_test |
| ner_oncology_response_to_treatment | ner_oncology_posology | ner_oncology | ner_oncology_limited_80p_for_benchmarks |
| ner_oncology_diagnosis | ner_oncology_demographics | ner_oncology_biomarker | ner_oncology_anatomy_granular | ner_oncology_anatomy_general |



</center>

In [ ]:
oncology_profiling_pipeline = nlp.PretrainedPipeline("ner_profiling_oncology", "en", "clinical/models")

ner_profiling_oncology download started this may take some time.
Approx size to download 2 GB
[OK!]


In [ ]:
text = """The had previously undergone a left mastectomy and an axillary lymph node dissection for a left breast cancer twenty years ago.
The tumor was positive for ER and PR. Postoperatively, radiotherapy was administered to the residual breast.
The cancer recurred as a right lung metastasis 13 years later. He underwent a regimen consisting of adriamycin (60 mg/m2) and cyclophosphamide (600 mg/m2) over six courses, as first line therapy."""

In [ ]:
oncology_result = oncology_profiling_pipeline.fullAnnotate(text)[0]
oncology_result.keys()

dict_keys(['ner_chunk_oncology_limited_80p_for_benchmarks', 'ner_oncology', 'ner_chunk_oncology_anatomy_general', 'document', 'ner_chunk_oncology_test', 'ner_chunk_oncology_tnm', 'ner_chunk_oncology', 'ner_chunk_oncology_therapy', 'ner_oncology_anatomy_general', 'ner_oncology_biomarker', 'ner_chunk_oncology_biomarker', 'ner_oncology_test', 'ner_oncology_response_to_treatment', 'token', 'ner_oncology_anatomy_granular', 'ner_oncology_therapy', 'ner_chunk_oncology_demographics', 'ner_chunk_oncology_response_to_treatment', 'ner_oncology_demographics', 'ner_oncology_diagnosis', 'ner_chunk_oncology_posology', 'ner_oncology_limited_80p_for_benchmarks', 'embeddings', 'ner_oncology_tnm', 'ner_chunk_oncology_diagnosis', 'ner_oncology_unspecific_posology', 'sentence', 'ner_chunk_oncology_unspecific_posology', 'ner_oncology_posology', 'ner_chunk_oncology_anatomy_granular'])

In [ ]:
def get_token_results(light_result):

    tokens = [j.result for j in light_result["token"]]
    sentences = [j.metadata["sentence"] for j in light_result["token"]]
    begins = [j.begin for j in light_result["token"]]
    ends = [j.end for j in light_result["token"]]
    model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

    for model_name in model_list:

        temp_df = pd.DataFrame(light_result[model_name])
        temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
        temp_df = temp_df[["jsl_label"]]

        # temp_df = get_ner_result(model_name)
        temp_df.columns = [model_name]
        df = pd.concat([df, temp_df], axis=1)

    # Filter columns to include only sentence, begin, end, token and all columns that start with 'ner_vop'
    filtered_df = df.loc[:, ['sentence', 'begin', 'end', 'token'] + [col for col in df.columns if col.startswith('ner_oncology')]]

    return filtered_df

In [ ]:
get_token_results(oncology_result)

sentence  begin  end       token       ner_oncology  \
0         0      0    2         The                  O   
1         0      4    6         had                  O   
2         0      8   17  previously                  O   
3         0     19   27   undergone                  O   
4         0     29   29           a                  O   
..      ...    ...  ...         ...                ...   
74        3    410  411          as                  O   
75        3    413  417       first  B-Line_Of_Therapy   
76        3    419  422        line  I-Line_Of_Therapy   
77        3    424  430     therapy                  O   
78        3    431  431           .                  O   

   ner_oncology_anatomy_general ner_oncology_biomarker ner_oncology_test  \
0                             O                      O                 O   
1                             O                      O                 O   
2                             O                      O                 O   
3                             O                      O                 O   
4                             O                      O                 O   
..                          ...                    ...               ...   
74                            O                      O                 O   
75                            O                      O                 O   
76                            O                      O                 O   
77                            O                      O                 O   
78                            O                      O                 O   

   ner_oncology_response_to_treatment ner_oncology_anatomy_granular  \
0                                   O                             O   
1                                   O                             O   
2                                   O                             O   
3                                   O                             O   
4                                   O                             O   
..                                ...                           ...   
74                                  O                             O   
75                  B-Line_Of_Therapy                             O   
76                  I-Line_Of_Therapy                             O   
77                                  O                             O   
78                                  O                             O   

   ner_oncology_therapy ner_oncology_demographics ner_oncology_diagnosis  \
0                     O                         O                      O   
1                     O                         O                      O   
2                     O                         O                      O   
3                     O                         O                      O   
4                     O                         O                      O   
..                  ...                       ...                    ...   
74                    O                         O                      O   
75    B-Line_Of_Therapy                         O                      O   
76    I-Line_Of_Therapy                         O                      O   
77                    O                         O                      O   
78                    O                         O                      O   

   ner_oncology_limited_80p_for_benchmarks ner_oncology_tnm  \
0                                        O                O   
1                                        O                O   
2                                        O                O   
3                                        O                O   
4                                        O                O   
..                                     ...              ...   
74                                       O                O   
75                       B-Line_Of_Therapy                O   
76                       I-Line_Of_Therapy                O 

# SDOH Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once.

- `ner_profiling_sdoh` : Returns results for sdoh NER models.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).





<center><b>NER Profiling SDOH Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| ner_sdoh | ner_sdoh_social_environment_wip | ner_sdoh_mentions |
| ner_sdoh_demographics_wip | ner_sdoh_community_condition_wip | ner_sdoh_substance_usage_wip | ner_sdoh_access_to_healthcare_wip |
| ner_sdoh_health_behaviours_problems_wip | ner_sdoh_income_social_status_wip |



</center>

In [ ]:
sdoh_profiling_pipeline = nlp.PretrainedPipeline("ner_profiling_sdoh", "en", "clinical/models")

ner_profiling_sdoh download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
text = """Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels. Smith works as a cleaning assistant and cannot access health insurance or paid sick leave. She has a son, a student at college. Pt with likely long-standing depression. She is aware she needs rehab. Pt reports having her catholic faith as a means of support as well.  She has a long history of etoh abuse, beginning in her teens. She reports she has been a daily drinker for 30 years, most recently drinking beer daily. She smokes a pack of cigarettes a day. She had DUI in April and was due to court this week."""

In [ ]:
sdoh_result = sdoh_profiling_pipeline.fullAnnotate(text)[0]
sdoh_result.keys()

dict_keys(['ner_chunk_ner_sdoh_mentions', 'ner_chunk_sdoh_health_behaviours_problems', 'document', 'ner_chunk_social_environment', 'ner_chunk_sdoh_access_to_healthcare', 'ner_chunk_sdoh_substance_usage', 'ner_sdoh_mentions', 'ner_sdoh_community_condition', 'ner_chunk_ner_demographics', 'ner_sdoh_demographics', 'ner_chunk_sdoh_community_condition', 'ner_sdoh', 'ner_chunk_sdoh_income_social_status', 'ner_social_environment', 'ner_sdoh_access_to_healthcare', 'ner_sdoh_health_behaviours_problems', 'token', 'ner_sdoh_substance_usage', 'embeddings', 'ner_sdoh_income_social_status', 'ner_chunk_sdoh', 'sentence'])

In [ ]:
def get_token_results(light_result):

    tokens = [j.result for j in light_result["token"]]
    sentences = [j.metadata["sentence"] for j in light_result["token"]]
    begins = [j.begin for j in light_result["token"]]
    ends = [j.end for j in light_result["token"]]
    model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

    for model_name in model_list:

        temp_df = pd.DataFrame(light_result[model_name])
        temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
        temp_df = temp_df[["jsl_label"]]

        # temp_df = get_ner_result(model_name)
        temp_df.columns = [model_name]
        df = pd.concat([df, temp_df], axis=1)

    # Filter columns to include only sentence, begin, end, token and all columns that start with 'ner_vop'
    filtered_df = df.loc[:, ['sentence', 'begin', 'end', 'token'] + [col for col in df.columns if col.startswith('ner_sdoh')]]

    return filtered_df

In [ ]:
get_token_results(sdoh_result)

sentence  begin  end  token ner_sdoh_mentions  \
0          0      0    4  Smith  B-sdoh_community   
1          0      6    7     is                 O   
2          0      9   10     55                 O   
3          0     12   16  years                 O   
4          0     18   20    old                 O   
..       ...    ...  ...    ...               ...   
159       12    817  818     to                 O   
160       12    820  824  court                 O   
161       12    826  829   this                 O   
162       12    831  834   week                 O   
163       12    835  835      .                 O   

    ner_sdoh_community_condition ner_sdoh_demographics ner_sdoh  \
0                              O                     O        O   
1                              O                     O        O   
2                              O                 B-Age    B-Age   
3                              O                 I-Age    I-Age   
4                              O                 I-Age    I-Age   
..                           ...                   ...      ...   
159                            O                     O        O   
160                            O                     O        O   
161                            O                     O        O   
162                            O                     O        O   
163                            O                     O        O   

    ner_sdoh_access_to_healthcare ner_sdoh_health_behaviours_problems  \
0                               O                                   O   
1                               O                                   O   
2                               O                                   O   
3                               O                                   O   
4                               O                                   O   
..                            ...                                 ...   
159                             O                                   O   
160                             O                                   O   
161                             O                                   O   
162                             O                                   O   
163                             O                                   O   

    ner_sdoh_substance_usage ner_sdoh_income_social_status  
0                          O                             O  
1                          O                             O  
2                          O                             O  
3                          O                             O  
4                          O                             O  
..                       ...                           ...  
159                        O                             O  
160                        O                             O  
161                        O                             O  
162                        O                             O  
163                        O                             O  

[164 rows x 12 columns]

# VOP Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once.

- `ner_profiling_vop` : Returns results for vop NER models.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).





<center><b>NER Profiling VOP Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| ner_vop_clinical_dept | ner_vop_temporal | ner_vop_test |
| ner_vop | ner_vop_problem | ner_vop_problem_reduced | ner_vop_treatment |
| ner_vop_demographic | ner_vop_anatomy |



</center>

In [ ]:
vop_profiling_pipeline = nlp.PretrainedPipeline("ner_profiling_vop", "en", "clinical/models")

ner_profiling_vop download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
text = """Hello, I am a 20-year-old woman who was diagnosed with hyperthyroidism around a month ago.For approximately four months, I've been experiencing symptoms such as feeling light-headed, battling poor digestion, dealing with anxiety attacks, depression, a sharp pain on my left side chest, an elevated heart rate, and a significant loss of weight. Due to these conditions, I was admitted to the hospital and just got discharged recently."""

In [ ]:
vop_result = vop_profiling_pipeline.fullAnnotate(text)[0]
vop_result.keys()

dict_keys(['ner_chunk_vop_problem_reduced', 'ner_vop_clinical_dept', 'ner_vop_temporal', 'ner_chunk_vop_test', 'document', 'ner_vop_test', 'ner_vop', 'ner_vop_problem', 'ner_vop_problem_reduced', 'ner_vop_treatment', 'ner_chunk_vop_problem', 'ner_chunk_vop', 'ner_vop_demographic', 'ner_chunk_vop_anatomy', 'ner_chunk_vop_clinical_dept', 'ner_chunk_vop_treatment', 'token', 'ner_chunk_vop_temporal', 'embeddings', 'ner_vop_anatomy', 'ner_chunk_vop_demographic', 'sentence'])

In [ ]:
def get_token_results(light_result):

    tokens = [j.result for j in light_result["token"]]
    sentences = [j.metadata["sentence"] for j in light_result["token"]]
    begins = [j.begin for j in light_result["token"]]
    ends = [j.end for j in light_result["token"]]
    model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

    for model_name in model_list:

        temp_df = pd.DataFrame(light_result[model_name])
        temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
        temp_df = temp_df[["jsl_label"]]

        # temp_df = get_ner_result(model_name)
        temp_df.columns = [model_name]
        df = pd.concat([df, temp_df], axis=1)

    # Filter columns to include only sentence, begin, end, token and all columns that start with 'ner_vop'
    filtered_df = df.loc[:, ['sentence', 'begin', 'end', 'token'] + [col for col in df.columns if col.startswith('ner_vop')]]

    return filtered_df

In [ ]:
get_token_results(vop_result)

sentence  begin  end       token ner_vop_clinical_dept ner_vop_temporal  \
0         0      0    4       Hello                     O                O   
1         0      5    5           ,                     O                O   
2         0      7    7           I                     O                O   
3         0      9   10          am                     O                O   
4         0     12   12           a                     O                O   
..      ...    ...  ...         ...                   ...              ...   
75        2    404  407        just                     O                O   
76        2    409  411         got                     O                O   
77        2    413  422  discharged  B-AdmissionDischarge                O   
78        2    424  431    recently                     O       B-DateTime   
79        2    432  432           .                     O                O   

   ner_vop_test               ner_vop ner_vop_problem ner_vop_problem_reduced  \
0             O                     O               O                       O   
1             O                     O               O                       O   
2             O                     O               O                       O   
3             O                     O               O                       O   
4             O                     O               O                       O   
..          ...                   ...             ...                     ...   
75            O                     O               O                       O   
76            O                     O               O                       O   
77            O  B-AdmissionDischarge               O                       O   
78            O            B-DateTime               O                       O   
79            O                     O               O                       O   

   ner_vop_treatment ner_vop_demographic ner_vop_anatomy  
0                  O                   O               O  
1                  O                   O               O  
2                  O                   O               O  
3                  O                   O               O  
4                  O                   O               O  
..               ...                 ...             ...  
75                 O                   O               O  
76                 O                   O               O  
77                 O                   O               O  
78                 O                   O               O  
79                 O                   O               O  

[80 rows x 13 columns]